#### What is OneHotEncoder?
class pyspark.ml.feature.OneHotEncoder(inputCols=None, outputCols=None, handleInvalid=’error’, dropLast=True, inputCol=None, outputCol=None) — One Hot Encoding is a technique for converting categorical attributes into a binary vector.

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index.

For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0].


The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [18]:
import findspark
findspark.init()

In [19]:

from pyspark.sql import SparkSession

In [20]:
spark = SparkSession.builder.appName('onehotencoder').getOrCreate()


In [21]:
from pyspark.sql.types import *

In [22]:
#create the structure of schema
schema = StructType().add("id","integer").add("name","string").add("qualification","string").add("age", "integer").add("gender", "string")

In [23]:
#create data
data = [
    (1,'John',"B.A.", 20, "Male"),
    (2,'Martha',"B.Com.", 20, "Female"),
    (3,'Mona',"B.Com.", 21, "Female"),
    (4,'Harish',"B.Sc.", 22, "Male"),
    (5,'Jonny',"B.A.", 22, "Male"),
    (6,'Maria',"B.A.", 23, "Female"),
    (7,'Monalisa',"B.A.", 21, "Female")
]

In [24]:
#create dataframe
df = spark.createDataFrame(data, schema=schema)
#columns of dataframe
df.columns

['id', 'name', 'qualification', 'age', 'gender']

In [25]:
df.show()

+---+--------+-------------+---+------+
| id|    name|qualification|age|gender|
+---+--------+-------------+---+------+
|  1|    John|         B.A.| 20|  Male|
|  2|  Martha|       B.Com.| 20|Female|
|  3|    Mona|       B.Com.| 21|Female|
|  4|  Harish|        B.Sc.| 22|  Male|
|  5|   Jonny|         B.A.| 22|  Male|
|  6|   Maria|         B.A.| 23|Female|
|  7|Monalisa|         B.A.| 21|Female|
+---+--------+-------------+---+------+



In [17]:
#df.distinct().show()

+---+--------+-------------+---+------+
| id|    name|qualification|age|gender|
+---+--------+-------------+---+------+
|  1|    John|         B.A.| 20|  Male|
|  2|  Martha|       B.Com.| 20|Female|
|  3|    Mona|       B.Com.| 21|Female|
|  4|  Harish|        B.Sc.| 22|  Male|
|  5|   Jonny|         B.A.| 22|  Male|
|  6|   Maria|         B.A.| 23|Female|
|  7|Monalisa|         B.A.| 21|Female|
+---+--------+-------------+---+------+



#### Apply OneHotEncoder to qualification and gender column
We can not apply OneHotEncoder to string columns directly. We need to first convert string columns to numeric value. For that we will use StringIndexer. After that we can apply OneHotEncoder.

In [26]:
#import required libraries
from pyspark.ml.feature import StringIndexer
qualification_indexer = StringIndexer(inputCol="qualification", outputCol="qualificationIndex")
#Fits a model to the input dataset with optional parameters.
df1 = qualification_indexer.fit(df).transform(df)
df1.show()

+---+--------+-------------+---+------+------------------+
| id|    name|qualification|age|gender|qualificationIndex|
+---+--------+-------------+---+------+------------------+
|  1|    John|         B.A.| 20|  Male|               0.0|
|  2|  Martha|       B.Com.| 20|Female|               1.0|
|  3|    Mona|       B.Com.| 21|Female|               1.0|
|  4|  Harish|        B.Sc.| 22|  Male|               2.0|
|  5|   Jonny|         B.A.| 22|  Male|               0.0|
|  6|   Maria|         B.A.| 23|Female|               0.0|
|  7|Monalisa|         B.A.| 21|Female|               0.0|
+---+--------+-------------+---+------+------------------+



Output: “B.A.” gets index 0 because it is the most frequent, then “B.Com” gets index 1 and “B.Sc.” gets index 2.

#### Apply StringIndexer to gender column


In [27]:
gender_indexer = StringIndexer(inputCol="gender", outputCol="genderIndex")
#Fits a model to the input dataset with optional parameters.
df2 = gender_indexer.fit(df).transform(df)
df2.show()

+---+--------+-------------+---+------+-----------+
| id|    name|qualification|age|gender|genderIndex|
+---+--------+-------------+---+------+-----------+
|  1|    John|         B.A.| 20|  Male|        1.0|
|  2|  Martha|       B.Com.| 20|Female|        0.0|
|  3|    Mona|       B.Com.| 21|Female|        0.0|
|  4|  Harish|        B.Sc.| 22|  Male|        1.0|
|  5|   Jonny|         B.A.| 22|  Male|        1.0|
|  6|   Maria|         B.A.| 23|Female|        0.0|
|  7|Monalisa|         B.A.| 21|Female|        0.0|
+---+--------+-------------+---+------+-----------+



#### Apply OneHotEncoder to qualificationIndex and genderIndex columns

In [28]:
from pyspark.ml.feature import OneHotEncoder

In [29]:
#onehotencoder to qualificationIndex
onehotencoder_qualification_vector = OneHotEncoder(inputCol="qualificationIndex", outputCol="qualification_vec")
df11 = onehotencoder_qualification_vector.fit(df1).transform(df1)

In [30]:
df11.show()

+---+--------+-------------+---+------+------------------+-----------------+
| id|    name|qualification|age|gender|qualificationIndex|qualification_vec|
+---+--------+-------------+---+------+------------------+-----------------+
|  1|    John|         B.A.| 20|  Male|               0.0|    (2,[0],[1.0])|
|  2|  Martha|       B.Com.| 20|Female|               1.0|    (2,[1],[1.0])|
|  3|    Mona|       B.Com.| 21|Female|               1.0|    (2,[1],[1.0])|
|  4|  Harish|        B.Sc.| 22|  Male|               2.0|        (2,[],[])|
|  5|   Jonny|         B.A.| 22|  Male|               0.0|    (2,[0],[1.0])|
|  6|   Maria|         B.A.| 23|Female|               0.0|    (2,[0],[1.0])|
|  7|Monalisa|         B.A.| 21|Female|               0.0|    (2,[0],[1.0])|
+---+--------+-------------+---+------+------------------+-----------------+



In [31]:
#onehotencoder to genderIndex
onehotencoder_gender_vector = OneHotEncoder(inputCol="genderIndex", outputCol="gender_vec")
df12 = onehotencoder_gender_vector.fit(df2).transform(df2)

In [32]:
df12.show()

+---+--------+-------------+---+------+-----------+-------------+
| id|    name|qualification|age|gender|genderIndex|   gender_vec|
+---+--------+-------------+---+------+-----------+-------------+
|  1|    John|         B.A.| 20|  Male|        1.0|    (1,[],[])|
|  2|  Martha|       B.Com.| 20|Female|        0.0|(1,[0],[1.0])|
|  3|    Mona|       B.Com.| 21|Female|        0.0|(1,[0],[1.0])|
|  4|  Harish|        B.Sc.| 22|  Male|        1.0|    (1,[],[])|
|  5|   Jonny|         B.A.| 22|  Male|        1.0|    (1,[],[])|
|  6|   Maria|         B.A.| 23|Female|        0.0|(1,[0],[1.0])|
|  7|Monalisa|         B.A.| 21|Female|        0.0|(1,[0],[1.0])|
+---+--------+-------------+---+------+-----------+-------------+



#### One hot encoding of a numeric column
#### Direct one-hot-encoding without indexing

In [33]:
onehotencoder_age_vector = OneHotEncoder(inputCol="age", outputCol="age_vec")
df13 = onehotencoder_age_vector.fit(df2).transform(df2)
df13.show()

+---+--------+-------------+---+------+-----------+---------------+
| id|    name|qualification|age|gender|genderIndex|        age_vec|
+---+--------+-------------+---+------+-----------+---------------+
|  1|    John|         B.A.| 20|  Male|        1.0|(23,[20],[1.0])|
|  2|  Martha|       B.Com.| 20|Female|        0.0|(23,[20],[1.0])|
|  3|    Mona|       B.Com.| 21|Female|        0.0|(23,[21],[1.0])|
|  4|  Harish|        B.Sc.| 22|  Male|        1.0|(23,[22],[1.0])|
|  5|   Jonny|         B.A.| 22|  Male|        1.0|(23,[22],[1.0])|
|  6|   Maria|         B.A.| 23|Female|        0.0|     (23,[],[])|
|  7|Monalisa|         B.A.| 21|Female|        0.0|(23,[21],[1.0])|
+---+--------+-------------+---+------+-----------+---------------+



#### Direct one-hot-encoding after indexing


In [ ]:
age_indexer = StringIndexer(inputCol="age", outputCol="ageIndex")
df13 = age_indexer.fit(df).transform(df)
onehotencoder_age_vector = OneHotEncoder(inputCol="ageIndex", outputCol="age_vec")
df14 = onehotencoder_age_vector.fit(df13).transform(df13)
df14.show()